# Script to create the SEN12 Multi-Temporal Urban Mapping Dataset
## Setup

In [ ]:
# import all packages
import rasterio
import json
from pathlib import Path
import math
import shutil
import matplotlib.pyplot as plt
import numpy as np
import ee

# initialize Google Earth Engine Python API (requires an authorized GEE account)
ee.Initialize()

# path to the SpaceNet 7 dataset: https://spacenet.ai/sn7-challenge/
sn7_path = Path('C:/Users/shafner/datasets/spacenet7')
# this folder should contain the metadata files for the train (metadata_train.json) and test (metadata_test.json) files
# and the Sentinel-1 SAR orbits file (s1_orbit_numbers.json).
# these files can be downloaded from the dataset folder in this git repository:
# https://github.com/SebastianHafner/SemiSupervisedMultiModalCD.git

# path to the SEN12 Multi-Temporal Urban Mapping Dataset
new_dataset_folder =  Path('C:/Users/shafner/datasets/multimodal_cd_dataset')

# define coordinate reference system
crs = 'EPSG:3857'

## Read and write functions for GeoTIFFs and JSON files

In [ ]:
# reading in geotiff file as numpy array
def read_tif(file: Path):
    if not file.exists():
        raise FileNotFoundError(f'File {file} not found')

    with rasterio.open(file) as dataset:
        arr = dataset.read()  # (bands X height X width)
        transform = dataset.transform
        crs = dataset.crs

    return arr.transpose((1, 2, 0)), transform, crs

def load_json(file: Path):
    with open(str(file)) as f:
        d = json.load(f)
    return d

def write_json(file: Path, data):
    with open(str(file), 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

## Get extent of the SpaceNet 7 study sites

In [ ]:
def get_sentinel_extent(dataset: str, aoi_id: str) -> ee.Feature:
    metadata_file = sn7_path / f'metadata_{dataset}.json'
    metadata = load_json(metadata_file)
    timeseries = metadata[aoi_id]
    year, month = timeseries[0]['year'], timeseries[0]['month']

    img_file = sn7_path / dataset / aoi_id / 'images_masked' / f'global_monthly_{year}_{month:02d}_mosaic_{aoi_id}.tif'
    img, transform, crs = read_tif(img_file)

    m, n, _ = img.shape
    x_res, _, x_min, _, y_res, y_max, *_ = transform

    x_max = x_min + n * x_res
    y_min = y_max + m * y_res
    x_min = int(math.ceil(x_min / 10) * 10)
    y_min = int(math.ceil(y_min / 10) * 10)

    n = int(x_max - x_min) // 10
    m = int(y_max - y_min) // 10
    x_max = x_min + n * 10
    y_max = y_min + m * 10
    
    crs == str(crs)
    assert(crs == 'EPSG:3857')
    
    bbox = ee.Geometry.Rectangle(
      coords=[x_min, y_min, x_max, y_max],
      proj=str(crs),
      geodesic=False,
    );
    
    return ee.Feature(bbox, {'aoi_id': aoi_id}).transform('EPSG:4326', 0.001)


dataset = 'train'

metadata_file = sn7_path / f'metadata_{dataset}.json'
metadata = load_json(metadata_file)
aoi_ids = metadata.keys()

extents = []
for i, aoi_id in enumerate(aoi_ids):
    extent = get_sentinel_extent(dataset, aoi_id)
    extents.append(extent)
    if i == 0:
        pass

fc = ee.FeatureCollection(extents)
dl_task = ee.batch.Export.table.toDrive(
    collection=fc,
    description='test',
    folder='sn7_extent',
    fileNamePrefix=f'sn7_sites_{dataset}',
    fileFormat='GeoJSON'
)
# dl_task.start()


fc = ee.FeatureCollection(extents)
dl_task = ee.batch.Export.table.toAsset(
    collection=fc,
    description=dataset,
    assetId=f'users/hafnersailing/sn7_sites_{dataset}'
)
# dl_task.start()

## Sentinel-1 SAR and Sentinel-2 MSI data pre-processing functions

In [ ]:
# Sentinel-2 MSI
def add_cloud_score(img: ee.Image) -> ee.Image:
    patch = ee.Geometry(img.get('patch'))
    cloud_probability = ee.Image(img.get('cloudProbability'))
    stats = cloud_probability.select('probability').reduceRegion(reducer=ee.Reducer.sum(),
                                                                 geometry=patch,
                                                                 scale=10,
                                                                 maxPixels=1e12)
    cloud_score = stats.get('probability')
    img = img.set('cloudScore', cloud_score)
    return img


def mostly_cloud_free_mosaic(patch: ee.Geometry, start_date: ee.Date, end_date: ee.Date) -> ee.Image:
    s2 = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(patch) \
        .map(lambda img: img.set('patch', patch))
    s2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
        .filterDate(start_date, end_date) \
        .filterBounds(patch)
    join_condition = ee.Filter.equals(leftField='system:index', rightField='system:index')
    s2 = ee.Join.saveFirst('cloudProbability').apply(primary=s2, secondary=s2_clouds, condition=join_condition)
    s2 = s2.map(add_cloud_score)
    s2 = ee.ImageCollection(s2)
    img = s2.sort('cloudScore', False).mosaic()
    img = img.unitScale(0, 10_000).clamp(0, 1)

    return img

# Sentinel-1 SAR
def single_orbit_mean(patch: ee.Geometry, start_date: ee.Date, end_date: ee.Date, orbit_number: int = None) -> ee.Image:

    # sup-setting data
    col = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(patch) \
        .filterDate(start_date, end_date) \
        .filterMetadata('instrumentMode', 'equals', 'IW') \
        .filterMetadata('transmitterReceiverPolarisation', 'equals', ['VV', 'VH'])

    # masking noise
    col = col.map(lambda img: img.updateMask(img.gte(-25)))
    # print(col.size().getInfo())

    # using orbit with more scenes if no orbit number is passed
    if orbit_number is None:
        asc_col = col.filterMetadata('orbitProperties_pass', 'equals', 'ASCENDING')
        # print('asc', asc_col.size().getInfo())
        desc_col = col.filterMetadata('orbitProperties_pass', 'equals', 'DESCENDING')
        # print('desc', desc_col.size().getInfo())
        col = ee.Algorithms.If(ee.Number(asc_col.size()).gt(desc_col.size()), asc_col, desc_col)
        col = ee.ImageCollection(col)
        # print(col.size().getInfo())
        if col.size().getInfo() == 0:
            return None

        # getting distinct orbit numbers
        orbit_numbers = col \
            .toList(col.size()) \
            .map(lambda img: ee.Number(ee.Image(img).get('relativeOrbitNumber_start'))) \
            .distinct().getInfo()

        # computing separate mean backscatter image for each orbit number
        means = ee.ImageCollection([])
        for number in orbit_numbers:
            mean = col.filterMetadata('relativeOrbitNumber_start', 'equals', number).mean()
            means = means.merge(ee.ImageCollection([mean]))

        img = means.mosaic()
    else:
        col = col.filterMetadata('relativeOrbitNumber_start', 'equals', orbit_number)
        if col.size().getInfo() == 0:
            return None
        img = col.mean()

    img = img.unitScale(-25, 0).clamp(0, 1)

    return img

## Upload the Sentinel data to Google Drive

In [ ]:
dataset = 'train'

orbit_numbers_file = sn7_path / 's1_orbit_numbers.json'
orbit_numbers = load_json(orbit_numbers_file)
metadata_file = sn7_path / f'metadata_{dataset}.json'
metadata = load_json(metadata_file)
aoi_ids = metadata.keys()

for i, aoi_id in enumerate(aoi_ids):
    timeseries = metadata[aoi_id]
    orbit_number = orbit_numbers[aoi_id] if aoi_id in orbit_numbers.keys() else None
    for timestamp in timeseries:
        year = timestamp['year']
        month = timestamp['month']
        start_date = ee.Date(f'{year}-{month:02d}-01')
        end_date = start_date.advance(1, 'month')
        extent = get_sentinel_extent(dataset, aoi_id)
        
        s2_img = mostly_cloud_free_mosaic(extent.geometry(), start_date, end_date)
        s2_dl_task = ee.batch.Export.image.toDrive(
            image=s2_img.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']).float(),
            region=extent.geometry().getInfo()['coordinates'],
            description=f'{aoi_id}_{year}_{month:02d}',
            folder=f'multimodal_cd_dataset_{dataset}',
            fileNamePrefix=f's2_{aoi_id}_{year}_{month:02d}',
            scale=10,
            crs=crs,
            maxPixels=250_000,
            fileFormat='GeoTIFF'
        )
        s2_dl_task.start()

        s1_img = single_orbit_mean(extent.geometry(), start_date, end_date, orbit_number)
        if s1_img is None:
            continue
        s1_dl_task = ee.batch.Export.image.toDrive(
            image=s1_img.select(['VV', 'VH']).float(),
            region=extent.geometry().getInfo()['coordinates'],
            description=f'{aoi_id}_{year}_{month:02d}',
            folder=f'multimodal_cd_dataset_{dataset}',
            fileNamePrefix=f's1_{aoi_id}_{year}_{month:02d}',
            scale=10,
            crs=crs,
            maxPixels=250_000,
            fileFormat='GeoTIFF'
        )
        s1_dl_task.start()

        if dataset == 'train':
            building_footprints = ee.FeatureCollection(f'users/hafnersailing/spacenet7/buildings_{aoi_id}')
            building_footprints = building_footprints \
                .filterMetadata('year', 'equals', year) \
                .filterMetadata('month', 'equals', month)
            # print(f'n buildings: {building_footprints.size().getInfo()}')
            building_footprints = building_footprints.map(lambda f: ee.Feature(f).set({'buildings': 1}))

            buildings = building_footprints.reduceToImage(['buildings'], ee.Reducer.first()) \
                .unmask() \
                .float() \
                .rename('buildings')

            building_percentage = buildings \
                .reproject(crs=crs, scale=1) \
                .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1000) \
                .reproject(crs=crs, scale=10) \
                .rename('buildingPercentage')

            bf_dl_task = ee.batch.Export.image.toDrive(
                image=building_percentage.float(),
                region=extent.geometry().getInfo()['coordinates'],
                description=f'{aoi_id}_{year}_{month:02d}',
                folder=f'multimodal_cd_dataset_{dataset}',
                fileNamePrefix=f'buildings_{aoi_id}_{year}_{month:02d}',
                scale=10,
                crs=crs,
                maxPixels=250_000,
                fileFormat='GeoTIFF'
            )
            bf_dl_task.start()

In [ ]:
dataset = 'train'

metadata_file = dataset_path / f'metadata_{dataset}.json'
metadata = load_json(metadata_file)
aoi_ids = metadata.keys()
files_path =  Path('C:/Users/shafner/datasets/multimodal_cd_dataset')

for i, aoi_id in enumerate(aoi_ids):
    timeseries = metadata[aoi_id]
    for timestamp in timeseries:
        year = timestamp['year']
        month = timestamp['month']
        features = ['s1', 's2', 'buildings']
        for feature in features:
            file = files_path / aoi_id / f'{feature}_{aoi_id}_{year}_{month:02d}.tif'
            if file.exists():
                folder = files_path / aoi_id / feature
                folder.mkdir(exist_ok=True)
                shutil.copy(file, folder)
                file.unlink()
            else:
                print(file.stem)


## Construct dataset from downloaded data
Make sure to place the downloaded datasets (multimodal_cd_dataset_train and multimodal_cd_dataset_test) in your new_dataset_path folder!

In [ ]:
dataset = 'train'

metadata_file = sn7_path / f'metadata_{dataset}.json'
metadata = load_json(metadata_file)
aoi_ids = metadata.keys()
files_path =  new_dataset_folder / f'multimodal_cd_dataset_{dataset}'

for i, aoi_id in enumerate(aoi_ids):
    timeseries = metadata[aoi_id]
    for timestamp in timeseries:
        year = timestamp['year']
        month = timestamp['month']
        features = ['s1', 's2', 'buildings']
        for feature in features:
            file = files_path / f'{feature}_{aoi_id}_{year}_{month:02d}.tif'
            if file.exists():
                folder_aoi = new_dataset_folder / aoi_id
                folder_aoi.mkdir(exist_ok=True)
                folder = folder_aoi / feature
                folder.mkdir(exist_ok=True)
                shutil.copy(file, folder)
                file.unlink()
            else:
                print(file.stem)

## Create metadata file

In [ ]:
bad_data = load_json(new_dataset_folder / 'bad_data.json')
new_metadata = {}
for dataset in ['train', 'test']:
    metadata_file = sn7_path / f'metadata_{dataset}.json'
    metadata = load_json(metadata_file)
    aoi_ids = metadata.keys()
    for i, aoi_id in enumerate(aoi_ids):
        timeseries = metadata[aoi_id]
        new_timeseries = []
        for j, timestamp in enumerate(timeseries):
            year = timestamp['year']
            month = timestamp['month']

            s1 = new_dataset_folder / aoi_id / 's1' / f's1_{aoi_id}_{year}_{month:02d}.tif'
            s2 = new_dataset_folder / aoi_id / 's2' / f's2_{aoi_id}_{year}_{month:02d}.tif'
            buildings = new_dataset_folder / aoi_id / 'buildings' / f'buildings_{aoi_id}_{year}_{month:02d}.tif'
            mask = sn7_path / dataset / aoi_id / 'UDM_masks' / f'global_monthly_{year}_{month:02d}_mosaic_{aoi_id}_UDM.tif'
            new_timestamp = {
                'aoi_id': aoi_id, 'year': year, 'month': month,
                's1': True if s1.exists() and not j in bad_data[aoi_id]['S1'] else False,
                's2': True if s2.exists() and not j in bad_data[aoi_id]['S2'] else False,
                'buildings': buildings.exists(),
                'masked': mask.exists(),
            }
            new_timeseries.append(new_timestamp)
        new_metadata[aoi_id] = new_timeseries

new_metadata_file = new_dataset_folder / 'metadata.json'
write_json(new_metadata_file, new_metadata)

In [ ]:
dataset_folder = Path('C:/Users/shafner/datasets/multimodal_cd_dataset')
metadata = load_json(dataset_folder / 'metadata.json')
data = {}
print(len(metadata.keys()))
for aoi_id in sorted(metadata.keys()):
    data[aoi_id] = {'S1': [], 'S2': []}

out_file = dataset_folder / 'bad_data.json'
write_json(out_file, data)